In [61]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from numpy  import array
from scipy import stats
from gensim.models.callbacks import CallbackAny2Vec
from sklearn import preprocessing

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity

# PREPROCESSING

In [13]:
corpus_filename = '../data/2nd_100.txt'

In [14]:
corpus = open(corpus_filename).read().splitlines()


In [15]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

In [16]:
def onlyEmojiSequences(tokens):
    threshold_emojis = [x for x in tokens if len(x) > 1]
    return threshold_emojis

In [17]:
tokenized_corpus = tokenize_corpus(corpus)
emojiSequences = onlyEmojiSequences(tokenized_corpus)

In [18]:
token_count = sum([len(sentence) for sentence in emojiSequences])
print("The corpus contains {0:,} emoji points.".format(token_count))

The corpus contains 367,398,481 emoji points.


# SETTING VARIABLES

In [93]:
# Dimensionality of the resulting word vectors.
num_features = 500

# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()
# num_workers = 1


# Context window length.
context_size = 4

# Downsample setting for frequent words.
downsampling = 1e-3

emoji2vec = w2v.Word2Vec(
    sg=0,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling,
    batch_words=1000,
    negative=20
)

class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    def __init__(self):
        self.epoch = 0
        self.tempLoss = 0
    def on_epoch_end(self, model):
        print("Epoch #{}  Loss-Value: {}".format(self.epoch, (emoji2vec.get_latest_training_loss() - self.tempLoss)))
        self.epoch += 1
        self.tempLoss = emoji2vec.get_latest_training_loss()
        
epoch_logger = EpochLogger()
emoji2vec.build_vocab(emojiSequences, progress_per=2)

# TRAINING

In [94]:
emoji2vec.train(emojiSequences, total_examples=emoji2vec.corpus_count, epochs = 4, compute_loss=True, callbacks=[epoch_logger])

Epoch #0  Loss-Value: 39139892.0
Epoch #1  Loss-Value: 22501848.0
Epoch #2  Loss-Value: 15030636.0
Epoch #3  Loss-Value: 11345984.0


(1100537908, 1469593924)

In [95]:
if not os.path.exists("trained"):
    os.makedirs("trained")
emoji2vec.save(os.path.join("trained", "CBOW_largeDS#7.w2v"))

# EVALUATING TRAINED MODEL

In [96]:
emoji2vec = w2v.Word2Vec.load(os.path.join("trained", "CBOW_largeDS#7.w2v"))

In [97]:
# extracts the 508 Annotator Results as the Gold-Standard
corpus_filename = '../data/EmoSim508.json'
corpus = open(corpus_filename).read()
annotator_similarity_score_508 = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# extract Wijeratne's Cosine_Similarities of the model which was trained on Google_Sense_Labels
google_sense_labels_score_508 = list(array(re.findall('(?<=Google_Sense_Label": )(.*?)(?=\,)', corpus)))

# glyph_pairs_1016
unicode_pairs_1016 = re.findall('(?<=unicodelong": "\\\)(.*?)(?=")', corpus)    
glyph_pairs_1016 = [codecs.decode(unicode_pairs_1016[x].replace(str('\\\\'),str('\\')).replace('_',''), 'unicode_escape') for x in range(len(unicode_pairs_1016))]

# computation of Cosine Similarity
goldstandard = []
selftrained = []
google_sense_labels = []
for x in range(len(annotator_similarity_score_508)):
    cosineSimilarity = None
    
    emoji1 = glyph_pairs_1016.pop(0)
    emoji2 = glyph_pairs_1016.pop(0)
    
    try:
        cosineSimilarity = emoji2vec.wv.similarity(emoji1, emoji2)
    except:
        print('the cosine similarity between ' + emoji1 + ' and ' + emoji2 + ' could not be computed.')
    
    if(cosineSimilarity is not None):
        goldstandard.append(annotator_similarity_score_508.pop(0))
        selftrained.append(cosineSimilarity)
        google_sense_labels.append(float(google_sense_labels_score_508.pop(0)))
    else:
        self.annotator_similarity_score_508.pop(0)
        float(self.google_sense_labels_score_508.pop(0))
        

# skalierter GoldStandard
min_max_scaler = preprocessing.MinMaxScaler()
scaled_goldstandard = min_max_scaler.fit_transform(np.asarray(goldstandard).reshape(-1, 1))

print()

# computation of SPEARRANK CORRELATION COEFFICIENT
meinSPEARMAN = stats.spearmanr(goldstandard, selftrained)
seinSPEARMAN = stats.spearmanr(goldstandard, google_sense_labels)
print('mein Spearman: {}'.format(meinSPEARMAN.correlation))
print('sein Spearman: {}'.format(seinSPEARMAN.correlation))


# computation of MAE
meinMAE = mean_absolute_error(scaled_goldstandard, selftrained)
seinMAE = mean_absolute_error(scaled_goldstandard, google_sense_labels)
print('mein MAE ist {}'.format(meinMAE))
print('sein MAE ist {}'.format(seinMAE))


# computation of MSE
meinMSE = mean_squared_error(scaled_goldstandard, selftrained)
seinMSE = mean_squared_error(scaled_goldstandard, google_sense_labels)
print('mein MSE ist {}'.format(meinMSE))
print('sein MSE ist {}'.format(seinMSE))



mein Spearman: 0.6434274539006912
sein Spearman: 0.761972416506743
mein MAE ist 0.1687345884155302
sein MAE ist 0.23962598425196854
mein MSE ist 0.04929642463594508
sein MSE ist 0.0789808562992126


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype <U4 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [19]:
print(len(annotator_similarity_score_508))

print(len(google_sense_labels_score_508))



10
10


In [ ]:
# experiment with some tests 
# what does he want? go to him with solid solid questions!!